In [133]:
import numpy as np
import pandas as pd
from scipy.linalg import block_diag
from sklearn.metrics import mean_squared_error

from stareg import Bspline, utils
import pprint

In [174]:
descr = ( ("s(1)", 20, "inc", 6000, "e"),  
          ("t(1,2)", (12,10), ("none", "none"), (6000,6000), ("e", "e")), )

In [128]:
model = dict()
parameter = ("type", "nr_splines", "constraint", "lambda_c", "knot_type")
for idx, submodel in enumerate(descr):
    model[f"f{idx+1}"] = dict()
    for type_and_value in zip(parameter, submodel):
        model[f"f{idx+1}"][type_and_value[0]] = type_and_value[1]
pprint.pprint(model)        

{'f1': {'constraint': 'inc',
        'knot_type': 'e',
        'lambda_c': 6000,
        'nr_splines': 10,
        'type': 's(1)'},
 'f2': {'constraint': ('none', 'none'),
        'knot_type': ('e', 'e'),
        'lambda_c': (6000, 6000),
        'nr_splines': (12, 10),
        'type': 't(1,2)'}}


In [175]:
n_data = 100
X = np.array([np.linspace(0,1,n_data), np.linspace(0,1,n_data)]).T
y = X[:,0]**2 - np.sin(X[:,1]*6) + np.random.normal(size=(n_data,))


In [181]:
s = stareg()
model = s.create_model_from_description(descr, X, y)

  0%|                                                                                                                                                                                              | 0/100 [00:00<?, ?it/s]

Class initialization
{'f1': {'constraint': 'inc',
        'knot_type': 'e',
        'lambda_c': 6000,
        'nr_splines': 20,
        'type': 's(1)'},
 'f2': {'constraint': ('none', 'none'),
        'knot_type': ('e', 'e'),
        'lambda_c': (6000, 6000),
        'nr_splines': (12, 10),
        'type': 't(1,2)'}}


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 45.32it/s]


In [177]:
pprint.pprint(model)

{'f1': {'B': array([[0.16666667, 0.66666667, 0.16666667, ..., 0.        , 0.        ,
        0.        ],
       [0.09470758, 0.63971157, 0.26473695, ..., 0.        , 0.        ,
        0.        ],
       [0.04717189, 0.56897307, 0.37710386, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.37710386, 0.56897307,
        0.04717189],
       [0.        , 0.        , 0.        , ..., 0.26473695, 0.63971157,
        0.09470758],
       [0.        , 0.        , 0.        , ..., 0.16666667, 0.66666667,
        0.16666667]]),
        'Dc': array([[-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  1.,  0.,  0.,  

In [180]:
class stareg():
    
    def __init__(self):
        print("Class initialization")
        self.BS = Bspline()
        
    def create_model_from_description(self, description, X, y):
        model = dict()
        parameter = ("type", "nr_splines", "constraint", "lambda_c", "knot_type")
        for idx, submodel in enumerate(descr):
            model[f"f{idx+1}"] = dict()
            for type_and_value in zip(parameter, submodel):
                model[f"f{idx+1}"][type_and_value[0]] = type_and_value[1]
        pprint.pprint(model) 
        for submodel in model:
            for key in model[submodel].keys():
                if key == "type":
                    type_ = model[submodel][key]
                    nr_splines = model[submodel]["nr_splines"]
                    knot_type = model[submodel]["knot_type"]
                    constraint = model[submodel]["constraint"]
                    lambda_c = model[submodel]["lambda_c"]
                    if type_.startswith("s"):
                        dim = int(type_[2])-1
                        data = X[:,dim]
                        order = 3
                        B, knots = self.BS.basismatrix(X=data, nr_splines=nr_splines, l=order, knot_type=knot_type).values()
                        Ds = utils.mm(nr_splines, constraint="smooth")
                        Dc = utils.mm(nr_splines, constraint=constraint)
                        lam = self.BS.calc_GCV(data, y, nr_splines=nr_splines, l=order, knot_type=knot_type, nr_lam=100, plot_=0)["best_lambda"]
                        coef_pls = BS.fit_Pspline(data, y, nr_splines=nr_splines, l=order, knot_type=knot_type, lam=lam)["coef_"]
                        W = utils.check_constraint(coef_pls, constraint, y=y, B=B)
                    elif type_.startswith("t"):
                        dim = [int(type_[2])-1,  int(type_[4])-1]
                        data = X[:,[dim[0],dim[1]]]
                        order = (3,3)
                        B, knots1, knots2 = self.BS.tensorproduct_basismatrix(X=data, nr_splines=nr_splines, l=order, knot_type=knot_type).values()
                        Ds1 = utils.mm(nr_splines, constraint="smooth", dim=0)
                        Ds2 = utils.mm(nr_splines, constraint="smooth", dim=1)
                        Dc1 = utils.mm(nr_splines, constraint=constraint[0], dim=0)                
                        Dc2 = utils.mm(nr_splines, constraint=constraint[0], dim=1)                
                        lam = self.BS.calc_GCV_2d(data, y, nr_splines=nr_splines, l=order, knot_type=knot_type, nr_lam=100, plot_=0)["best_lambda"]
                        coef_pls = BS.fit_Pspline(data, y, nr_splines=nr_splines, l=order, knot_type=knot_type, lam=lam)["coef_"]
                        W1 = utils.check_constraint_dim1(coef_pls, constraint[0], nr_splines)
                        W2 = utils.check_constraint_dim2(coef_pls, constraint[0], nr_splines)

                        knots, Ds, Dc, W = dict(), dict(), dict(), dict()
                        knots["k1"], knots["k2"] = knots1, knots2
                        Ds["Ds1"], Ds["Ds2"] = Ds1, Ds2
                        Dc["Dc1"], Dc["Dc2"] = Dc1, Dc2
                        W["v1"], W["v2"] = W1, W2

            model[submodel]["B"] = B
            model[submodel]["knots"] = knots
            model[submodel]["Ds"] = Ds
            model[submodel]["Dc"] = Dc
            model[submodel]["knots"] = knots
            model[submodel]["weights"] = W
            model[submodel]["coef_pls"] = coef_pls
            model[submodel]["best_lambda"] = lam
        self.model = model
        return model

In [182]:
model["f1"].keys()

dict_keys(['type', 'nr_splines', 'constraint', 'lambda_c', 'knot_type', 'B', 'knots', 'Ds', 'Dc', 'weights', 'coef_pls', 'best_lambda'])